In [ ]:
import pandas as pd
import requests
from tabulate import tabulate
import xml.etree.ElementTree as ET


In [3]:
def authenticate_tableau(tableau_server_config):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']
    token_name = config['personal_access_token_name']
    token_secret = config['personal_access_token_secret']
    site_name = config['site_name']

    # Authenticate and get the token
    auth_url = f'{server}/api/{api_version}/auth/signin'
    auth_payload = {
        'credentials': {
            'personalAccessTokenName': token_name,
            'personalAccessTokenSecret': token_secret,
            'site': {
                'contentUrl': site_name,
            }
        }
    }

    # Make the authentication request
    auth_response = requests.post(auth_url, json=auth_payload)

    # Parse the XML response
    root = ET.fromstring(auth_response.content)

    # Get the token value
    token = root.find('.//{http://tableau.com/api}credentials').get('token')

    # Get the site ID
    site_id = root.find('.//{http://tableau.com/api}credentials/{http://tableau.com/api}site').get('id')

    # Get the user ID
    user_id = root.find('.//{http://tableau.com/api}credentials/{http://tableau.com/api}user').get('id')

    return token, site_id, user_id

In [173]:
def get_site_users(tableau_server_config, site_id, user_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']

    # Construct the URL for the GET request to get the user information
    user_url = f'{server}/api/{api_version}/sites/{site_id}/users/{user_id}'

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the user information
    response = requests.get(user_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        ns = {'ts': 'http://tableau.com/api'}  # Define the namespace

        user = root.find('ts:user', ns)
        if user is not None:
            user_info = {
                'authSetting': user.get('authSetting'),
                'email': user.get('email'),
                'externalAuthUserId': user.get('externalAuthUserId'),
                'fullName': user.get('fullName'),
                'id': user.get('id'),
                'lastLogin': user.get('lastLogin'),
                'name': user.get('name'),
                'siteRole': user.get('siteRole'),
                'locale': user.get('locale'),
            }
            df = pd.DataFrame([user_info])  # Wrap the dictionary in a list
            return df
        else:
            print("Error: User not found in response.")
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [174]:
def query_workbooks_for_site(tableau_server_config, site_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']

    # Construct the URL for the GET request to get the workbook information
    view_url = f'{server}/api/{api_version}/sites/{site_id}/workbooks'

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the workbook information
    response = requests.get(view_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)

        # Find the workbooks element
        workbooks_element = root.find('{http://tableau.com/api}workbooks')

        # Create a list to store the workbook information
        workbooks_info = []

        # Iterate over the workbook elements and extract the information
        for workbook_element in workbooks_element.findall('{http://tableau.com/api}workbook'):
            workbook_info = {
                'id': workbook_element.get('id'),
                'name': workbook_element.get('name'),
                'contentUrl': workbook_element.get('contentUrl'),
                'webpageUrl': workbook_element.get('webpageUrl'),
                'showTabs': workbook_element.get('showTabs'),
                'size': workbook_element.get('size'),
                'createdAt': workbook_element.get('createdAt'),
                'updatedAt': workbook_element.get('updatedAt'),
                'encryptExtracts': workbook_element.get('encryptExtracts'),
                'defaultViewId': workbook_element.get('defaultViewId'),
                'projectId': workbook_element.find('{http://tableau.com/api}project').get('id'),
                'locationId': workbook_element.find('{http://tableau.com/api}location').get('id'),
                'ownerId': workbook_element.find('{http://tableau.com/api}owner').get('id')
            }
            workbooks_info.append(workbook_info)

        # Convert the workbook information into a DataFrame
        df = pd.DataFrame(workbooks_info)
        return df
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [183]:
def download_workbook(tableau_server_config, site_id, workbook_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']

    # Construct the URL for the GET request to get the workbook information
    workbook_url = f'{server}/api/{api_version}/sites/{site_id}/workbooks/{workbook_id}/content'
    print("Workbook URL: ", workbook_url)

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the workbook information
    response = requests.get(workbook_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        print("Response Content: ", response.content)
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [176]:
def get_site_views(tableau_server_config, site_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']
    site_views_url = f'{server}/api/{api_version}/sites/{site_id}/views'

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the view information
    response = requests.get(site_views_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)

        # Extract the views from the XML
        views = root.find('.//{http://tableau.com/api}views')

        # Create a list to store the view data
        view_data = []

        # Iterate over the views and extract the relevant information
        for view in views.findall('{http://tableau.com/api}view'):
            view_info = {
                'id': view.get('id'),
                'name': view.get('name'),
                'contentUrl': view.get('contentUrl'),
                'createdAt': view.get('createdAt'),
                'updatedAt': view.get('updatedAt'),
                'viewUrlName': view.get('viewUrlName'),
                'workbookId': view.find('{http://tableau.com/api}workbook').get('id'),
                'ownerId': view.find('{http://tableau.com/api}owner').get('id'),
                'projectId': view.find('{http://tableau.com/api}project').get('id'),
                'locationId': view.find('{http://tableau.com/api}location').get('id'),
                'locationType': view.find('{http://tableau.com/api}location').get('type')
            }
            view_data.append(view_info)

        # Create a DataFrame from the view data
        df = pd.DataFrame(view_data)
        return df
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [177]:
def get_site_view(tableau_server_config, site_id, view_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']

    # Construct the URL for the GET request to get the view information
    view_url = f'{server}/api/{api_version}/sites/{site_id}/views/{view_id}'

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the view information
    response = requests.get(view_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)

        # Extract the view information from the XML
        view = root.find('{http://tableau.com/api}view')
        view_info = {
            'id': view.get('id'),
            'name': view.get('name'),
            'contentUrl': view.get('contentUrl'),
            'createdAt': view.get('createdAt'),
            'updatedAt': view.get('updatedAt'),
            'viewUrlName': view.get('viewUrlName'),
            'workbookId': view.find('{http://tableau.com/api}workbook').get('id'),
            'ownerId': view.find('{http://tableau.com/api}owner').get('id'),
            'projectId': view.find('{http://tableau.com/api}project').get('id'),
            'locationId': view.find('{http://tableau.com/api}location').get('id'),
            'locationType': view.find('{http://tableau.com/api}location').get('type')
        }

        # Convert the view information into a DataFrame
        df = pd.DataFrame([view_info])
        return df
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [178]:
def get_crosstab(tableau_server_config, site_id, view_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']

    # Construct the URL for the GET request to get the user information
    user_url = f'{server}/api/{api_version}/sites/{site_id}/views/{view_id}/crosstab/excel'

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the user information
    response = requests.get(user_url, headers=headers)

    save_path = 'crosstab.xlsx'

    # Check if the request was successful
    if response.status_code == 200:
        print("Content successfully retrieved")

        # Save the content as an Excel file
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"File saved to {save_path}")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [179]:
def query_view_data(tableau_server_config, site_id, view_id, token):
    # Extract the configuration details
    config = tableau_server_config['tableau_prod']
    server = config['server']
    api_version = config['api_version']
    # Construct the URL for the GET request to get the view information
    view_url = f'{server}/api/{api_version}/sites/{site_id}/views/{view_id}/data'

    # Set up the headers with the authentication token
    headers = {
        'X-Tableau-Auth': token
    }

    # Make the GET request to retrieve the view information
    response = requests.get(view_url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        print(response.content)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    

In [ ]:
# Testing starts

In [4]:
# Define the Tableau server configuration
tableau_server_config = {
    'tableau_prod': {
        'server': 'https://prod-xyz.online.tableau.com/',
        'api_version': '3.22',
        'personal_access_token_name': 'my token',
        'personal_access_token_secret': 'abcd1234',
        'site_name': 'my site',
        'site_url': ''
    }
}

# Authenticate and get the token, site ID, and user ID
token, site_id, user_id = authenticate_tableau(tableau_server_config)

# Print the values
print("Token:", token)
print("Site ID:", site_id)
print("User ID:", user_id)

ConnectionError: HTTPSConnectionPool(host='prod-xyz.online.tableau.com', port=443): Max retries exceeded with url: //api/3.22/auth/signin (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001CE687EFA10>: Failed to resolve 'prod-xyz.online.tableau.com' ([Errno 11001] getaddrinfo failed)"))

In [5]:
user_df = get_site_users(tableau_server_config, site_id, user_id, token)

NameError: name 'get_site_users' is not defined

In [6]:
user_df

NameError: name 'user_df' is not defined

In [7]:
workbooks_df  = query_workbooks_for_site(tableau_server_config, site_id, token)

tabular_format = tabulate(workbooks_df, headers='keys', tablefmt='psql')

# Open a file for writing
with open('workbooks.txt', 'w', encoding='utf-8') as file:
    # Write the tabular data to the file
    file.write(tabular_format)

NameError: name 'query_workbooks_for_site' is not defined

In [8]:
workbooks_df

NameError: name 'workbooks_df' is not defined

In [9]:
workbook_id = "fde09d1c-a3df-4315-a029-40774dba181d"
download_workbook(tableau_server_config, site_id, workbook_id, token)

NameError: name 'download_workbook' is not defined

In [10]:
site_views_df = get_site_views(tableau_server_config, site_id, token)

# # Convert the DataFrame to a string in a tabular format
# tabular_data = site_views_df.to_string(index=False)

# # Open a file for writing
# with open('site_views.txt', 'w', encoding='utf-8') as file:
#     # Write the tabular data to the file
#     file.write(tabular_data)

tabular_format = tabulate(site_views_df, headers='keys', tablefmt='psql')

# Open a file for writing
with open('site_views.txt', 'w', encoding='utf-8') as file:
    # Write the tabular data to the file
    file.write(tabular_format)


NameError: name 'get_site_views' is not defined

In [11]:
site_views_df

NameError: name 'site_views_df' is not defined

In [12]:
merged_df = pd.merge(workbooks_df, site_views_df, left_on='id', right_on='workbookId', how='inner')

tabular_format = tabulate(merged_df, headers='keys', tablefmt='psql')

# Open a file for writing
with open('merge.txt', 'w', encoding='utf-8') as file:
    # Write the tabular data to the file
    file.write(tabular_format)

NameError: name 'workbooks_df' is not defined

In [13]:
merged_df

NameError: name 'merged_df' is not defined

In [14]:
# for _, row in site_views_df.iterrows():
#     print("\n ----------------------")
#     print(row)

In [15]:
view_id = "29210604-e7c0-4503-bfc9-222d0f2b7f93"
site_view_df = get_site_view(tableau_server_config, site_id, view_id, token)

NameError: name 'get_site_view' is not defined

In [16]:
site_view_df

NameError: name 'site_view_df' is not defined

In [17]:
summary_view_id = "29210604-e7c0-4503-bfc9-222d0f2b7f93"
new_prescriber_details_view_id = "6640604a-4e0f-4b8a-bb33-23f33afca26b"
get_crosstab(tableau_server_config, site_id, "c374351a-09a3-4a92-9700-f24dc711a808", token)

NameError: name 'get_crosstab' is not defined

In [18]:
# url = "https://prod-useast-b.online.tableau.com//api/3.22/sites/656be640-fdc3-4b44-a3da-9346aa9a848c/views/29210604-e7c0-4503-bfc9-222d0f2b7f93/crosstab/excel"
# requests.get(url=url, headers={'Authorization': token})

In [19]:
view_id = "29210604-e7c0-4503-bfc9-222d0f2b7f93"
view_data_df = query_view_data(tableau_server_config, site_id, view_id, token)

NameError: name 'query_view_data' is not defined